In [1]:
import numpy
import imp
from re import sub
import logging
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import os
try:
    changed
except NameError:
    os.chdir('..')
    os.system('find -name *.pyc | xargs rm')
    changed = True

import sys
stdout = sys.stdout
from hyperstream import HyperStream, UTC, TimeInterval, __version__
sys.stdout = stdout

    
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set_style("dark")

minute = timedelta(minutes=1)
t1 = datetime(2016, 4, 28, 20, 0, 0, 0, UTC)
t2 = t1 + minute
ti = TimeInterval(t1, t2)

module_file_old = '/Users/td6301/dev/IRC-SPHERE_public/HyperStream/hyperstream/tools/percentiles_from_list/2016-12-02_v0.0.1.py'
module_file_new = '/Users/td6301/dev/IRC-SPHERE_public/HyperStream/hyperstream/tools/percentiles_from_list/2017-07-18_v0.0.2.py'

print(__version__)

0.3.0-beta


In [2]:
hs = HyperStream()

In [3]:
def load_module(module_file):
    module_file_components = module_file[:-3].split('/')

    with open(module_file, 'rb') as fp:
        module_name = '_'.join(map(lambda pp: sub(r'[^a-zA-Z0-9]', '_', pp), module_file_components))

        if module_name in sys.modules:
            print("module {} already loaded ...".format(module_name))
            del sys.modules[module_name]

        mod = imp.load_module(
            module_name, fp, module_file,
            ('.py', 'rb', imp.PY_SOURCE)
        )

        return mod

In [4]:
# Generate some random date
dg = hs.plugins.data_generators

ticker = hs.channel_manager.memory.get_or_create_stream('ticker')
random = hs.channel_manager.memory.get_or_create_stream('random')
hs.tools.clock().execute(sources=[], sink=ticker, interval=ti)
dg.tools.random(seed=1234).execute(sources=[], sink=random, interval=ti, alignment_stream=ticker)

print(random.window().values()[:5])

[0.9664535356921388, 0.4407325991753527, 0.007491470058587191, 0.9109759624491242, 0.939268997363764]


In [10]:
def print_percentiles(stream_old, stream_new):
    data_old = stream_old.window().items()
    data_new = stream_new.window().items()
    
    for i, (old, new) in enumerate(zip(data_old, data_new)):
        print('{0:2.0f}, {1:2.0f}, {2}, {3}, {4}'.format(
            old.timestamp.minute, old.timestamp.second, old.value, new.value, np.array_equal(old.value, new.value)))
        if not np.array_equal(old.value, new.value):
            raise

In [11]:
# Force loading of tool modules
mod_old = load_module(module_file_old)
mod_new = load_module(module_file_new)

q = [25, 50, 75]

p_old = mod_old.PercentilesFromList(percentiles=q)
p_new = mod_new.PercentilesFromList(percentiles=q)

perc_old = hs.channel_manager.memory.get_or_create_stream('perc_old')
perc_new = hs.channel_manager.memory.get_or_create_stream('perc_new')

hs.channel_manager.memory.purge_stream(perc_old.stream_id)
hs.channel_manager.memory.purge_stream(perc_new.stream_id)

p_old.execute(sources=[random], sink=perc_old, interval=ti)
p_new.execute(sources=[random], sink=perc_new, interval=ti)

print_percentiles(perc_old, perc_new)

module _Users_td6301_dev_IRC_SPHERE_public_HyperStream_hyperstream_tools_percentiles_from_list_2016_12_02_v0_0_1 already loaded ...
module _Users_td6301_dev_IRC_SPHERE_public_HyperStream_hyperstream_tools_percentiles_from_list_2017_07_18_v0_0_2 already loaded ...
 0,  1, [0.9664535356921388, 0.9664535356921388, 0.9664535356921388], [0.9664535356921388, 0.9664535356921388, 0.9664535356921388], True
 0,  2, [0.4407325991753527, 0.4407325991753527, 0.4407325991753527], [0.4407325991753527, 0.4407325991753527, 0.4407325991753527], True
 0,  3, [0.007491470058587191, 0.007491470058587191, 0.007491470058587191], [0.007491470058587191, 0.007491470058587191, 0.007491470058587191], True
 0,  4, [0.9109759624491242, 0.9109759624491242, 0.9109759624491242], [0.9109759624491242, 0.9109759624491242, 0.9109759624491242], True
 0,  5, [0.939268997363764, 0.939268997363764, 0.939268997363764], [0.939268997363764, 0.939268997363764, 0.939268997363764], True
 0,  6, [0.5822275730589491, 0.58222757305894

In [12]:
# Force loading of tool modules
mod_old = load_module(module_file_old)
mod_new = load_module(module_file_new)

p_old = mod_old.PercentilesFromList(n_segments=11)
p_new = mod_new.PercentilesFromList(n_segments=11)

perc_old = hs.channel_manager.memory.get_or_create_stream('perc_old')
perc_new = hs.channel_manager.memory.get_or_create_stream('perc_new')

hs.channel_manager.memory.purge_stream(perc_old.stream_id)
hs.channel_manager.memory.purge_stream(perc_new.stream_id)

p_old.execute(sources=[random], sink=perc_old, interval=ti)
p_new.execute(sources=[random], sink=perc_new, interval=ti)

print_percentiles(perc_old, perc_new)


module _Users_td6301_dev_IRC_SPHERE_public_HyperStream_hyperstream_tools_percentiles_from_list_2016_12_02_v0_0_1 already loaded ...
module _Users_td6301_dev_IRC_SPHERE_public_HyperStream_hyperstream_tools_percentiles_from_list_2017_07_18_v0_0_2 already loaded ...
 0,  1, [0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388], [0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388, 0.9664535356921388], True
 0,  2, [0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.4407325991753527, 0.44073259